In [1]:
import pyspark as spark
from pyspark import SparkContext
# initialize a new Spark Context to use for the execution of the script
sc = SparkContext(appName="MY-APP-NAME", master="local[*]")
# prevent useless logging messages
sc.setLogLevel("ERROR")

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor, LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.classification import FMClassifier
import numpy as np
from pyspark.mllib.clustering import KMeans
import matplotlib.pyplot as plt
from pyspark.sql.functions import col
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
import matplotlib.pyplot as plt

spark = SparkSession.builder \
    .master("local") \
    .appName("appName") \
    .getOrCreate()

In [3]:
def performance_metrics(predicted_values):
    evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

    #Root Mean Square Error
    rmse = evaluator.evaluate(predicted_values)
    print("RMSE: %.3f" % rmse)

    # Mean Square Error
    mse = evaluator.evaluate(predicted_values, {evaluator.metricName: "mse"})
    print("MSE: %.3f" % mse)

    # Mean Absolute Error
    mae = evaluator.evaluate(predicted_values, {evaluator.metricName: "mae"})
    print("MAE: %.3f" % mae)

    # r2 - coefficient of determination
    r2 = evaluator.evaluate(predicted_values, {evaluator.metricName: "r2"})
    print("r2: %.3f" %r2)

In [4]:
data = spark.read.options(header='true', inferschema='true', delimiter=',').csv("data/df.csv")
data.show(5)

+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|  R|  C|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|        q_pressure|
+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148| 50| 10| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503| 1.712044550044049| 4.787830588906654|
|       463| 50| 10| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.452311677836793|14.032851614737082|
|       471| 20| 20|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|2.9888939958281715|  7.08478856513968|
|       496| 20| 20| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 2.781362935991756| 18.63630522318597|
|      1238| 50| 50| -4.046307639606464|34.68797

# Regression #1 
## Regress m and q parameters for the pressure attribute indepentently

## Regression m_pressure


In [5]:
features = ['breath_ids', 'R', 'C', 'm_u_in', 'q_u_in', 'm_u_out', 'q_u_out']
lr_data = data.select(col('m_pressure').alias("label"), *features)

In [6]:
lr_data = lr_data.select([col(c).cast("double").alias(c) for c in lr_data.columns])

In [7]:
train, test = lr_data.randomSplit([0.8, 0.2],seed=69)
print("We have %d training examples and %d test examples." % (train.count(), test.count()))

We have 13702 training examples and 3534 test examples.


In [8]:
from pyspark.ml.feature import VectorAssembler
for_prediction = features.copy()
for_prediction.remove('breath_ids')
vectorAssembler = VectorAssembler(inputCols=for_prediction, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")

In [9]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(labelCol="label")

In [10]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

paramGrid = ParamGridBuilder()\
.addGrid(gbt.maxBins, (2, 10))\
.addGrid(gbt.maxDepth, list(np.arange(2, 28, 2)))\
.addGrid(gbt.maxIter, (2, 50))\
.build()

evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())
tvs = TrainValidationSplit(estimator=gbt,
                           estimatorParamMaps=paramGrid,
                           evaluator=RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2"), 
                           trainRatio=0.8)

In [11]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, standardScaler, tvs])

In [12]:
model = pipeline.fit(train)

In [13]:
predictions = model.transform(test)

In [16]:
performance_metrics(predictions)

RMSE: 1.339
MSE: 1.794
MAE: 0.984
r2: 0.747


### Get predicted m to the original dataset

In [17]:
m_pressure = model.transform(lr_data)
m_press_predicted = m_pressure.select([col("breath_ids").alias("b_id"), 'prediction'])

## Regression for q_pressure

In [18]:
lr_data = data.select(col('q_pressure').alias("label"), *features)

In [19]:
train, test = lr_data.randomSplit([0.8, 0.2], seed=69)

In [20]:
vectorAssembler = VectorAssembler(inputCols=for_prediction, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
pipeline = Pipeline(stages=[vectorAssembler, standardScaler, tvs])

### Test performances on q_pressure

In [21]:
pipelineModel = pipeline.fit(train)

In [22]:
predictions = pipelineModel.transform(test)

In [23]:
performance_metrics(predictions)

RMSE: 1.480
MSE: 2.189
MAE: 1.043
r2: 0.944


### Get predicted q to the original dataset 

In [24]:
q_pressure = pipelineModel.transform(lr_data)
q_press_predicted = q_pressure.select([col("breath_ids").alias("b_id"), 'prediction'])

# Output from the first regression technique

In [25]:
df_regressed_1 = data.select(features)
df_regressed_1 = df_regressed_1.join(m_press_predicted, m_press_predicted.b_id == df_regressed_1.breath_ids).select(*df_regressed_1.columns, col('prediction').alias('m_pressure'))
df_regressed_1 = df_regressed_1.join(q_press_predicted, q_press_predicted.b_id == df_regressed_1.breath_ids).select(*df_regressed_1.columns, col('prediction').alias('q_pressure'))
df_regressed_1.show(5)

+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|  R|  C|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|        q_pressure|
+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148| 50| 10| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503|3.0137361594490844| 5.702867519125647|
|       463| 50| 10| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.616595676139938|14.964707095430349|
|       471| 20| 20|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|2.6657098188435597| 8.284327979404408|
|       496| 20| 20| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 4.528314059710676|20.582469248426573|
|      1238| 50| 50| -4.046307639606464|34.68797

# Regression #2
### 1. Regress the m parameter for pressure
### 2. Include the predicted value of m in the training set when predicting the q

In [26]:
# Append to the original dataset the m_pressure field calculated from the step before
df_m = data.select(features)
df_m = data.join(m_press_predicted, m_press_predicted.b_id == df_m.breath_ids).select(*df_m.columns, col('prediction').alias('m_pressure'))
df_m.show(5)

+----------+---+---+-------------------+------------------+------------------+------------------+------------------+
|breath_ids|  R|  C|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|
+----------+---+---+-------------------+------------------+------------------+------------------+------------------+
|       148| 50| 10| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503|3.0137361594490844|
|       463| 50| 10| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.616595676139938|
|       471| 20| 20|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|2.6657098188435597|
|       496| 20| 20| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 4.528314059710676|
|      1238| 50| 50| -4.046307639606464|34.687972375171306|3.5981683352545537|-4.872868892058087|1.8439833579824747|
+----------+---+---+-------------------+------------------+-----

In [27]:
m_features = ['breath_ids', 'R', 'C', 'm_u_in', 'q_u_in', 'm_u_out', 'q_u_out', 'm_pressure']
q = data.select('q_pressure', col('breath_ids').alias('b_id'))
lr_data = df_m.join(q, q.b_id == df_m.breath_ids).select(*df_m.columns, col('q_pressure').alias("label"))
lr_data.show(5)

+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|  R|  C|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|             label|
+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148| 50| 10| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503|3.0137361594490844| 4.787830588906654|
|       463| 50| 10| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.616595676139938|14.032851614737082|
|       471| 20| 20|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|2.6657098188435597|  7.08478856513968|
|       496| 20| 20| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 4.528314059710676| 18.63630522318597|
|      1238| 50| 50| -4.046307639606464|34.68797

In [28]:
train, test = lr_data.randomSplit([0.8, 0.2], seed=69)

In [29]:
featuresCols = m_features.copy()
featuresCols.remove('breath_ids')
vectorAssembler = VectorAssembler(inputCols=for_prediction, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
pipeline = Pipeline(stages=[vectorAssembler, standardScaler, tvs])

## Regression for q_pressure

In [30]:
model = pipeline.fit(train)

In [31]:
predictions = model.transform(test)

In [32]:
performance_metrics(predictions)

RMSE: 1.503
MSE: 2.260
MAE: 1.035
r2: 0.941


## Output from the second regression technique


In [33]:
q_pressure = model.transform(lr_data)
q_press_predicted = q_pressure.select([col("breath_ids").alias("b_id"), 'prediction'])

In [34]:
df_regressed_2 = data.select(m_features)
df_regressed_2 = df_regressed_2.join(q_press_predicted, q_press_predicted.b_id == df_regressed_2.breath_ids).select(*df_regressed_2.columns, col('prediction').alias('q_pressure'))
df_regressed_2.show(5)

+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|  R|  C|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        m_pressure|        q_pressure|
+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148| 50| 10| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503| 1.712044550044049|  5.34503881585481|
|       463| 50| 10| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274| 5.452311677836793|14.948109371520614|
|       471| 20| 20|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|2.9888939958281715|   7.7907105734523|
|       496| 20| 20| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 2.781362935991756| 20.42425504277897|
|      1238| 50| 50| -4.046307639606464|34.68797

# Regression #3 
### 1. Regress the q parameter for pressure 
### 2. Include the predicted value of q in the trianning set when predicting the m

In [35]:
# Append to the original dataset the q_pressure field calculated from the step before
df_q = data.select(features)
df_q = df_q.join(q_press_predicted, q_press_predicted.b_id == df_m.breath_ids).select(*df_q.columns, col('prediction').alias('q_pressure'))
df_q.show(5)

+----------+---+---+-------------------+------------------+------------------+------------------+------------------+
|breath_ids|  R|  C|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        q_pressure|
+----------+---+---+-------------------+------------------+------------------+------------------+------------------+
|       148| 50| 10| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503|  5.34503881585481|
|       463| 50| 10| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274|14.948109371520614|
|       471| 20| 20|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|   7.7907105734523|
|       496| 20| 20| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 20.42425504277897|
|      1238| 50| 50| -4.046307639606464|34.687972375171306|3.5981683352545537|-4.872868892058087|23.395317476086657|
+----------+---+---+-------------------+------------------+-----

In [36]:
q_features = ['breath_ids', 'R', 'C', 'm_u_in', 'q_u_in', 'm_u_out', 'q_u_out', 'q_pressure']
m = data.select('m_pressure', col('breath_ids').alias('b_id'))
lr_data = df_q.join(m, m.b_id == df_q.breath_ids).select(*df_q.columns, col('m_pressure').alias('label'))
lr_data.show(5)

+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|  R|  C|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        q_pressure|             label|
+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148| 50| 10| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503|  5.34503881585481| 1.712044550044049|
|       463| 50| 10| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274|14.948109371520614| 5.452311677836793|
|       471| 20| 20|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|   7.7907105734523|2.9888939958281715|
|       496| 20| 20| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 20.42425504277897| 2.781362935991756|
|      1238| 50| 50| -4.046307639606464|34.68797

In [37]:
train, test = lr_data.randomSplit([0.8, 0.2], seed=69)

In [38]:
for_prediction = q_features.copy()
for_prediction.remove('breath_ids')
vectorAssembler = VectorAssembler(inputCols=for_prediction, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
pipeline = Pipeline(stages=[vectorAssembler, standardScaler, tvs])

## Regression for m_pressure

In [39]:
model = pipeline.fit(train)
prediction = model.transform(test)

In [40]:
performance_metrics(prediction)

RMSE: 1.318
MSE: 1.738
MAE: 0.956
r2: 0.743


## Output from the second regression technique

In [41]:
m_pressure = model.transform(lr_data)
m_press_predicted = m_pressure.select([col("breath_ids").alias("b_id"), 'prediction'])

In [42]:
df_regressed_3 = data.select(q_features)
df_regressed_3 = df_regressed_3.join(m_press_predicted, m_press_predicted.b_id == df_regressed_3.breath_ids).select(*df_regressed_3.columns, col('prediction').alias('m_pressure'))
df_regressed_3.show(5)

+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|breath_ids|  R|  C|             m_u_in|            q_u_in|           m_u_out|           q_u_out|        q_pressure|        m_pressure|
+----------+---+---+-------------------+------------------+------------------+------------------+------------------+------------------+
|       148| 50| 10| -0.627586555937512| 3.279187855644647| 3.583992459539968|-4.387057482814503| 4.787830588906654|2.9105266268096273|
|       463| 50| 10| -2.730341695467989|11.496348651324867|3.5980943797891185|-4.875283862177274|14.032851614737082| 5.600717415261689|
|       471| 20| 20|-1.1012242363556006|10.255015772333854| 3.589597364806948|-4.547209213742605|  7.08478856513968| 4.109787015938288|
|       496| 20| 20| -17.10135369945869| 54.39589825365896|3.5900412844154244| -4.54881661950235| 18.63630522318597| 4.493247356099403|
|      1238| 50| 50| -4.046307639606464|34.68797